In [1]:
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [2]:
data = pd.read_excel('/content/Ringkasan Saham-20231204.xlsx')

In [3]:
data.to_csv("data_saham.csv", index=False, encoding="utf-8")

In [4]:
data_saham = pd.read_csv('/content/data_saham.csv')

In [5]:
data_saham.head()

,No,Kode Saham,Nama Perusahaan,Remarks,Sebelumnya,Open Price,Tanggal Perdagangan Terakhir,First Trade,Tertinggi,Terendah,...,Bid,Bid Volume,Listed Shares,Tradeble Shares,Weight For Index,Foreign Sell,Foreign Buy,Non Regular Volume,Non Regular Value,Non Regular Frequency
0,1,AALI,Astra Agro Lestari Tbk.,--S-18AE106000D232------------,7475,0,04 Des 2023,0,7475,7125,...,7125,104000,1924688333,1924688333,390711732,1011200,69100,1200,8760000,1
1,2,ABBA,Mahaka Media Tbk.,--U-4100000000E614-E---------X,67,0,04 Des 2023,0,71,67,...,69,142400,3935892857,3935892857,1342139464,3000,1140100,0,0,0
2,3,ABDA,Asuransi Bina Dana Arta Tbk.,--U-4105000000G412-----------X,5900,0,04 Des 2023,0,0,0,...,0,0,620806680,620806680,31474899,0,0,0,0,0
3,4,ABMM,ABM Investama Tbk.,--S-1805000000A121------------,3390,0,04 Des 2023,0,3430,3370,...,3390,103600,2753165000,2753165000,568803889,459400,458300,0,0,0
4,5,ACES,Ace Hardware Indonesia Tbk.,--SO1S75MD0000E743------------,735,730,04 Des 2023,730,750,720,...,720,3802500,17150000000,17150000000,6832560000,15108100,25577600,8165024,5993127616,9


In [6]:
print('Total Saham:', len(data_saham['Kode Saham'].unique()))
print('Total Harga Tertinggi', len(data_saham['Tertinggi']))
print('Total Harga Terendah', len(data_saham['Terendah']))
print('Total Harga Penutupan', len(data_saham['Penutupan']))

Total Saham: 905
Total Harga Tertinggi 905
Total Harga Terendah 905
Total Harga Penutupan 905


In [7]:
data_saham.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 905 entries, 0 to 904
Data columns (total 28 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   No                            905 non-null    int64  
 1   Kode Saham                    905 non-null    object 
 2   Nama Perusahaan               905 non-null    object 
 3   Remarks                       905 non-null    object 
 4   Sebelumnya                    905 non-null    int64  
 5   Open Price                    905 non-null    int64  
 6   Tanggal Perdagangan Terakhir  905 non-null    object 
 7   First Trade                   905 non-null    int64  
 8   Tertinggi                     905 non-null    int64  
 9   Terendah                      905 non-null    int64  
 10  Penutupan                     905 non-null    int64  
 11  Selisih                       905 non-null    int64  
 12  Volume                        905 non-null    int64  
 13  Nilai

In [8]:
data_saham.describe()

,No,Sebelumnya,Open Price,First Trade,Tertinggi,Terendah,Penutupan,Selisih,Volume,Nilai,...,Bid,Bid Volume,Listed Shares,Tradeble Shares,Weight For Index,Foreign Sell,Foreign Buy,Non Regular Volume,Non Regular Value,Non Regular Frequency
count,905.000000,905.000000,905.000000,905.000000,905.000000,905.000000,905.000000,905.000000,9.050000e+02,9.050000e+02,...,905.000000,9.050000e+02,9.050000e+02,9.050000e+02,9.050000e+02,9.050000e+02,9.050000e+02,9.050000e+02,9.050000e+02,905.000000
mean,453.000000,1211.745856,220.997790,221.188950,1030.499448,988.953591,1225.929282,14.183425,3.407521e+07,1.309250e+10,...,1074.851934,2.168439e+06,1.249778e+10,1.175585e+10,4.117747e+09,5.279713e+06,5.720773e+06,2.053338e+06,1.391515e+09,1.961326
std,261.395295,3630.062567,1567.310036,1568.487853,2924.763238,2744.457871,3748.431153,283.506539,5.558789e+08,8.488610e+10,...,3228.637220,3.414987e+07,4.721114e+10,4.530623e+10,2.915455e+10,9.880131e+07,9.584193e+07,1.949470e+07,1.582136e+10,33.060243
min,1.000000,2.000000,0.000000,0.000000,0.000000,0.000000,2.000000,-600.000000,0.000000e+00,0.000000e+00,...,0.000000,0.000000e+00,3.600000e+06,3.600000e+06,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000
25%,227.000000,78.000000,0.000000,0.000000,57.000000,53.000000,79.000000,-2.000000,2.010000e+04,4.951000e+06,...,59.000000,3.000000e+02,1.307735e+09,1.159200e+09,2.113660e+08,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000
50%,453.000000,244.000000,0.000000,0.000000,192.000000,185.000000,244.000000,0.000000,5.196000e+05,1.152216e+08,...,193.000000,1.430000e+04,3.410475e+09,3.128140e+09,6.421823e+08,1.500000e+03,0.000000e+00,0.000000e+00,0.000000e+00,0.000000
75%,679.000000,820.000000,0.000000,0.000000,745.000000,715.000000,815.000000,2.000000,6.677500e+06,1.876937e+09,...,735.000000,2.069000e+05,9.936339e+09,9.610000e+09,2.362785e+09,2.951000e+05,4.116000e+05,0.000000e+00,0.000000e+00,0.000000
max,905.000000,51500.000000,24600.000000,24600.000000,48075.000000,40075.000000,51500.000000,8000.000000,1.660633e+10,1.808320e+12,...,48075.000000,1.013286e+09,1.201410e+12,1.150838e+12,8.386481e+11,2.963200e+09,2.857252e+09,3.587960e+08,3.938126e+11,987.000000


### Cleaning data

In [9]:
cols = ['No', 'Remarks', 'Sebelumnya', 'Open Price', 'First Trade', 'Selisih', 'Volume', 'Nilai', 'Frekuensi', 'Index Individual', 'Offer', 'Offer Volume', 'Bid', 'Bid Volume', 'Listed Shares', 'Tradeble Shares', 'Weight For Index',
        'Foreign Sell', 'Foreign Buy', 'Non Regular Volume', 'Non Regular Value', 'Non Regular Frequency']
data_saham.drop(cols, axis=1, inplace=True)
data_saham.isnull().sum()

Kode Saham                      0
Nama Perusahaan                 0
Tanggal Perdagangan Terakhir    0
Tertinggi                       0
Terendah                        0
Penutupan                       0
dtype: int64

In [10]:
data_saham.head()

,Kode Saham,Nama Perusahaan,Tanggal Perdagangan Terakhir,Tertinggi,Terendah,Penutupan
0,AALI,Astra Agro Lestari Tbk.,04 Des 2023,7475,7125,7125
1,ABBA,Mahaka Media Tbk.,04 Des 2023,71,67,69
2,ABDA,Asuransi Bina Dana Arta Tbk.,04 Des 2023,0,0,5900
3,ABMM,ABM Investama Tbk.,04 Des 2023,3430,3370,3390
4,ACES,Ace Hardware Indonesia Tbk.,04 Des 2023,750,720,720


In [ ]:
# data_saham = data_saham.set_index('Kode Saham')
# data_saham.index

In [11]:
# memperbaiki nama atribut data_saham frame
data_saham.rename(columns={
    'Kode Saham'                : 'kode_saham',
    'Nama Perusahaan'          : 'nama_perusahaan',
    'Tanggal Perdagangan Terakhir'         : 'tanggal_perdagangan_terakhir',
    'Tertinggi' : 'tertinggi',
    'Terendah'           : 'terendah',
    'Penutupan'         : 'penutupan'
}, inplace=True)

data_saham.head(3)

,kode_saham,nama_perusahaan,tanggal_perdagangan_terakhir,tertinggi,terendah,penutupan
0,AALI,Astra Agro Lestari Tbk.,04 Des 2023,7475,7125,7125
1,ABBA,Mahaka Media Tbk.,04 Des 2023,71,67,69
2,ABDA,Asuransi Bina Dana Arta Tbk.,04 Des 2023,0,0,5900


### Membuat kolom baru dengan perhitungan mean
mean disini tujuan nya untuk kesamaan (similarity degree) antara data mean dari hasil forecasting supaya mendapatkan kode saham dari data latih

In [12]:
data_saham["hasil_mean"] = data_saham.apply(lambda x: (x["tertinggi"] + x["terendah"] + x["penutupan"]) / 3, axis=1)

In [13]:
data_saham.head()

,kode_saham,nama_perusahaan,tanggal_perdagangan_terakhir,tertinggi,terendah,penutupan,hasil_mean
0,AALI,Astra Agro Lestari Tbk.,04 Des 2023,7475,7125,7125,7241.666667
1,ABBA,Mahaka Media Tbk.,04 Des 2023,71,67,69,69.000000
2,ABDA,Asuransi Bina Dana Arta Tbk.,04 Des 2023,0,0,5900,1966.666667
3,ABMM,ABM Investama Tbk.,04 Des 2023,3430,3370,3390,3396.666667
4,ACES,Ace Hardware Indonesia Tbk.,04 Des 2023,750,720,720,730.000000


In [14]:
data_saham.isnull().sum()

kode_saham                      0
nama_perusahaan                 0
tanggal_perdagangan_terakhir    0
tertinggi                       0
terendah                        0
penutupan                       0
hasil_mean                      0
dtype: int64

In [15]:
data_saham.duplicated().sum()

0

### Model Content-based Recommendation
Merekomendasikan item yang mirip dengan item similarity degree. Content-based filtering akan mempelajari data saham pada mean berdasarkan objek data similarity degree.

In [16]:
# Inisialisasi TfidfVectorizer
tfv = TfidfVectorizer()
# Melakukan perhitungan idf pada data cuisine
tfv.fit(data_saham['hasil_mean'].astype(str))

TfidfVectorizer()

Selanjutnya, melakukan fit dan transformasi atribut data saham yaitu hasil_mean ke dalam bentuk matriks.

In [17]:
# Melakukan fit lalu ditransformasikan ke bentuk matrix
tfidf_matrix = tfv.fit_transform(data_saham['hasil_mean'].astype(str))

# Melihat ukuran matrix tfidf
tfidf_matrix.shape

(905, 569)

Matriks yang di miliki berukuran (905, 569). Nilai 905 merupakan ukuran data saham dan 569 merupakan matrik data hasil_mean.

Untuk menghasilkan vektor tf-idf dalam bentuk matriks, menggunakan fungsi [todense()](https://docs.scipy.org/doc/scipy/reference/generated/scipy.sparse.csc_matrix.todense.html)

In [18]:
tfidf_matrix.todense()

matrix([[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]])

In [19]:

# Membuat dataframe untuk melihat tf-idf matrix
# Kolom diisi dengan hasil mean
# Baris diisi dengan kode saham

pd.DataFrame(
    tfidf_matrix.todense(),
    columns = tfv.get_feature_names_out(),
    index   = data_saham.kode_saham
).sample(20, axis=1).sample(10, axis=0)

,563,795,37,1298,8075,3440,33,456,290,5866,230,261,63,4343,505,9766,193,2486,1345,91
kode_saham,,,,,,,,,,,,,,,,,,,,
ITIC,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
KINO,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
BTON,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
JAWA,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
MREI,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
OBMD,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
AGII,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
JARR,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
MKTR,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### Cosine Similiarity
Selanjutnya, menghitung derajat kesamaan (similarity degree) antar hasil mean dengan teknik cosine similarity menggunakan fungsi [cosine_similarity](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.pairwise.cosine_similarity.html) dari library sklearn.

In [20]:
# Menghitung cosine similarity pada matrix tf-idf
cosine_sim = cosine_similarity(tfidf_matrix)
cosine_sim

array([[1.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 1.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 1.        , ..., 0.        , 0.20583235,
        0.        ],
       ...,
       [0.        , 0.        , 0.        , ..., 1.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.20583235, ..., 0.        , 1.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        1.        ]])

In [21]:
# Membuat dataframe dari variabel cosine_sim dengan baris dan kolom berupa kode saham
cosine_sim_df = pd.DataFrame(
    cosine_sim,
    columns=data_saham['kode_saham'],
    index=data_saham['kode_saham']
)

print('Shape:', cosine_sim_df.shape)

# Melihat similarity matrix pada setiap hasil mean
cosine_sim_df.sample(5, axis=1).sample(10, axis=0)

Shape: (905, 905)


kode_saham,FMII,SMRA,VTNY,BABY,BUKK
kode_saham,,,,,
PEVE,0.000000,0.0,0.0,0.0,0.000000
LPKR,0.000000,0.0,0.0,0.0,0.000000
CTBN,0.000000,0.0,0.0,0.0,0.000000
WINS,0.000000,0.0,0.0,0.0,0.215648
AIMS,0.190531,0.0,0.0,0.0,0.000000
TRUE,0.000000,0.0,0.0,0.0,0.000000
PSAB,0.000000,0.0,0.0,0.0,0.000000
AGRO,0.000000,0.0,0.0,0.0,0.205832
TOWR,0.000000,0.0,0.0,0.0,0.000000


### Recommendation Testing

In [22]:
cosine_sim_df

kode_saham,AALI,ABBA,ABDA,ABMM,ACES,ACST,ADCP,ADES,ADHI,ADMF,...,WSKT,WTON,YELO,YPAS,YULE,ZATA,ZBRA,ZINC,ZONE,ZYRX
kode_saham,,,,,,,,,,,,,,,,,,,,,
AALI,1.0,0.0,0.000000,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
ABBA,0.0,1.0,0.000000,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
ABDA,0.0,0.0,1.000000,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.205832,0.0
ABMM,0.0,0.0,0.000000,1.0,0.0,0.00000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
ACES,0.0,0.0,0.000000,0.0,1.0,0.00000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ZATA,0.0,0.0,0.000000,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.000000,0.0
ZBRA,0.0,0.0,0.000000,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.000000,0.0
ZINC,0.0,0.0,0.000000,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.000000,0.0


Membuat fungsi saham_recommendations untuk menampilkan data kode saham yang direkomendasikan oleh algoritma sistem yang telah dibuat, dengan parameter masukan berupa kode saham yang akan di dinisiasi pada similarity degree.

In [23]:

def saham_recommendations(hasil_mean, similarity_data=cosine_sim_df, items=data_saham[['kode_saham', 'hasil_mean']], k=10):
    # Mengambil data dengan menggunakan argpartition untuk melakukan partisi secara tidak langsung sepanjang sumbu yang diberikan
    # Dataframe diubah menjadi numpy
    # Range(start, stop, step)
    index = similarity_data.loc[:,hasil_mean].to_numpy().argpartition(range(-1, -k, -1))
    # Mengambil data dengan similarity terbesar dari index yang ada
    closest = similarity_data.columns[index[-1:-(k+2):-1]]
    # Drop nama saham agar saham yang dicari tidak muncul dalam daftar rekomendasi
    closest = closest.drop(hasil_mean, errors='ignore')

    return pd.DataFrame(closest).merge(items).head(k)


In [24]:
data_saham['kode_saham']

0      AALI
1      ABBA
2      ABDA
3      ABMM
4      ACES
       ... 
900    ZATA
901    ZBRA
902    ZINC
903    ZONE
904    ZYRX
Name: kode_saham, Length: 905, dtype: object

In [25]:
data_saham['hasil_mean']

0      7241.666667
1        69.000000
2      1966.666667
3      3396.666667
4       730.000000
          ...     
900      50.000000
901     459.333333
902      50.000000
903    1116.666667
904     179.333333
Name: hasil_mean, Length: 905, dtype: float64

In [26]:
# Inisialisasi variabel baru
# 16000 ini kataknlah pengeluaran user hari ini lalu di bagi dengan seminggu
var = 16000 / 7

# Hitung perbedaan
# Perbedaam antara mean output forecasting dengan seluruh dataset saham pada kolom mean
diff = var - data_saham["hasil_mean"]

print(diff)

# Cari baris yang sesuai
idx = diff <= 0.1

# Hasil pada kolom hasil_mean dengan value True mengindikasi data cocok pada idx
print(idx)
print(idx.sum())

0     -4955.952381
1      2216.714286
2       319.047619
3     -1110.952381
4      1555.714286
          ...     
900    2235.714286
901    1826.380952
902    2235.714286
903    1169.047619
904    2106.380952
Name: hasil_mean, Length: 905, dtype: float64
0       True
1      False
2      False
3       True
4      False
       ...  
900    False
901    False
902    False
903    False
904    False
Name: hasil_mean, Length: 905, dtype: bool
97


menampilkan dataframe dengan kondisional true

In [27]:
data_saham[idx]

,kode_saham,nama_perusahaan,tanggal_perdagangan_terakhir,tertinggi,terendah,penutupan,hasil_mean
0,AALI,Astra Agro Lestari Tbk.,04 Des 2023,7475,7125,7125,7241.666667
3,ABMM,ABM Investama Tbk.,04 Des 2023,3430,3370,3390,3396.666667
7,ADES,Akasha Wira International Tbk.,04 Des 2023,10375,10000,10250,10208.333333
9,ADMF,Adira Dinamika Multi Finance Tbk.,04 Des 2023,10750,10600,10650,10666.666667
12,ADRO,Adaro Energy Indonesia Tbk.,04 Des 2023,2610,2550,2560,2573.333333
...,...,...,...,...,...,...,...
836,TPIA,Chandra Asri Petrochemical Tbk.,04 Des 2023,2940,2860,2910,2903.333333
859,UNIC,Unggul Indah Cahaya Tbk.,04 Des 2023,8075,8075,8075,8075.000000
863,UNTR,United Tractors Tbk.,04 Des 2023,22675,22200,22425,22433.333333
864,UNVR,Unilever Indonesia Tbk.,04 Des 2023,3670,3550,3630,3616.666667


In [28]:
# Mencari data dengan kolom mean terendah
# karena hasil pada idx berjumlah banyak, maka ambil data dengan hasil minimal
actual_df = data_saham[idx]
data_selected = actual_df['hasil_mean'].min()

In [29]:
data_selected

2380.0

In [30]:
saham_selected = data_saham[data_saham.hasil_mean.eq(data_selected)]
saham_selected

,kode_saham,nama_perusahaan,tanggal_perdagangan_terakhir,tertinggi,terendah,penutupan,hasil_mean
773,SONA,Sona Topas Tourism Industry Tbk.,04 Des 2023,2380,2380,2380,2380.0


mengambil data kode saham dengan Cosine Similiarity pada data output forecasting tadi

In [31]:
data_selected_saham = saham_selected.loc[:, 'kode_saham'].to_string(index=False)

In [32]:
data_selected_saham

'SONA'

### Hasil rekomendasi

In [33]:
saham_recommendations(data_selected_saham).drop_duplicates()

,kode_saham,hasil_mean
0,ZYRX,179.333333
1,FOLK,64.000000
2,FOOD,118.333333
3,FORU,158.333333
4,FMII,349.333333
5,GAMA,6.000000
6,FREN,52.000000
7,FORZ,16.666667
8,FUTR,50.000000
9,FPNI,196.333333


10 Saham (sesuai banyak k) yang direkomendasikan bersesuaian dengan kode saham dari Cosine Similiarity

akurasi nya 10/10 * 100 = 100%

### Test deploy

In [34]:
# Simpan model
import pickle

with open('model.pkl', 'wb') as f:
    pickle.dump(cosine_sim_df, f)

In [35]:
# Muat model
with open('model.pkl', 'rb') as f:
    cosine_sim_df_deploy = pickle.load(f)

In [36]:
saham_recommendations(data_selected_saham, similarity_data=cosine_sim_df_deploy).drop_duplicates()

,kode_saham,hasil_mean
0,ZYRX,179.333333
1,FOLK,64.000000
2,FOOD,118.333333
3,FORU,158.333333
4,FMII,349.333333
5,GAMA,6.000000
6,FREN,52.000000
7,FORZ,16.666667
8,FUTR,50.000000
9,FPNI,196.333333


### Test flask

In [37]:
!pip install flask

In [38]:
!pip install pyngrok

In [39]:
!ngrok authtoken 2Z5qKCbKRO5vainLb7Rwi2vWF9s_2sPbwVc5KEaNQt4yZwoFz

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [40]:
global data_selected_saham
# Import library
from flask import Flask
from pyngrok import ngrok
import json

import os
import threading

# Inisialisasi Flask
app = Flask(__name__)
port = "5000"

public_url = ngrok.connect(port).public_url
print(" * ngrok tunnel \"{}\" -> \"http://127.0.0.1:{}\"".format(public_url, port))

@app.route('/')
def index():
  return 'hello world'


@app.route('/saham')
def recommendations():
    # Muat model
    with open('model.pkl', 'rb') as f:
        cosine_sim_df = pickle.load(f)

    # Lakukan prediksi
    recommendations = saham_recommendations(data_selected_saham, similarity_data=cosine_sim_df)

    # Kembalikan hasil prediksi
    return json.dumps(recommendations.to_dict(orient='records'))

# Jalankan Flask
if __name__ == '__main__':
    threading.Thread(target=app.run, kwargs={"use_reloader": False}).start()

 * ngrok tunnel "https://b0ab-34-83-39-197.ngrok-free.app" -> "http://127.0.0.1:5000"
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
